# VGG116
![](https://picgogogo.oss-cn-hangzhou.aliyuncs.com/img/vgg16.jpg)

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

# 其次加载其他模块
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD

import cv2
import h5py as h5py
import numpy as np

# 加载字体库作为训练样本
from tensorflow.keras.datasets import mnist

In [ ]:
# 原版的VGG16
model_vgg = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

for layer in model_vgg.layers:
    layer.trainable = False#别去调整之前的卷积层的参数
    
model = Flatten()(model_vgg.output)
model = Dense(100, activation='relu', name='fc1')(model)
model = Dense(100, activation='relu', name='fc2')(model)
model = Dropout(0.5)(model)
model = Dense(10, activation='softmax', name='prediction')(model)
model_vgg_mnist_pretrain = Model(model_vgg.input, model, name='vgg16_pretrain')

sgd = SGD(lr=0.05, decay=1e-5) 

model_vgg_mnist_pretrain.compile(loss='categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])

(X_train, y_train), (X_test, y_test) = mnist.load_data()
# X_train, y_train = X_train[:1000], y_train[:1000]    #训练集1000条
# X_test, y_test = X_test[:100], y_test[:100]          #测试集100条

print(X_train.shape, y_train.shape)

def y_onehot(y):
    y_o = np.zeros(10)
    y_o[y] = 1
    return y_o

# 转换图像
X_train_new = np.array([cv2.cvtColor(cv2.resize(img, (224, 224)), cv2.COLOR_GRAY2RGB) for img in X_train])
X_test_new = np.array([cv2.cvtColor(cv2.resize(img, (224, 224)), cv2.COLOR_GRAY2RGB) for img in X_test])
y_train_new = np.array([y_onehot(y_train[i]) for i in range(len(y_train))])
y_test_new = np.array([y_onehot(y_train[i]) for i in range(len(y_test))])

print(X_train_new.shape, y_train_new.shape)

In [ ]:
# 训练
model_vgg_mnist_pretrain.fit(X_train_new, y_train_new, epochs=50, batch_size=25)

# 精确度
pred = model_vgg_mnist_pretrain.predict(X_test_new, batch_size=15)
accuracy = np.mean(np.argmax(pred, axis=1) == np.argmax(y_test_new, axis=1))
print(accuracy)